## Request

Dans un programme réalisé dans le langage de votre choix (de préférence C, C++, Java ou Python), réalisez au moins trois requêtes et faites afficher les résultats de façon lisible et compréhensible pour un non informaticien : liste des départements d'une région donnée, liste des communes de plus de X habitants d'un département donné, la région la plus/la moins peuplée, les communes les plus/les moins peuplées d'un département, etc.

In [37]:
import psycopg2
import psycopg2.extras
import pandas as pd
from db import connect
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

conn = connect()
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [38]:
import drop
import my_create_table

# creation de tables

drop.drop_tables(cur, conn)
my_create_table.create_tables(cur, conn)

Tables supprimées avec succès
Tables créées avec succès


In [39]:
import insert_commune
import insert_stats_population
import insert_stats_mariage

insert_commune.fill_tables_commune()
insert_stats_population.fill_tables_population() #30s-1min
insert_stats_mariage.fill_tables_mariage() 

Données insérées dans la table 'region' avec succès
Données insérées dans la table 'departement' avec succès
Données insérées dans la table 'commune' avec succès
Données insérées dans la table 'chef_lieu_departement' avec succès
Données insérées dans la table 'chef_lieu_region' avec succès
Données insérées dans la table 'statistiques_population' avec succès
Données insérées dans la table 'statistiques_mariages_age' avec succès
Données insérées dans la table 'statistiques_mariages_etat_matrimonial' avec succès
Données insérées dans la table 'statistiques_mariages_mensuel' avec succès
Données insérées dans la table 'statistiques_mariages_origine' avec succès


### Liste des départements d'une Region

In [40]:
nom_region = "Nouvelle-Aquitaine"

request = f"""
SELECT d.id_departement, d.nom_departement
FROM departement d
JOIN region r ON d.id_region = r.id_region
WHERE r.nom_region = '{nom_region}';
"""

cur.execute(request)
rows = cur.fetchall()

df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement'])
print(df)


   id_departement       nom_departement
0              16              Charente
1              17     Charente-Maritime
2              19               Corrèze
3              23                Creuse
4              24              Dordogne
5              33               Gironde
6              40                Landes
7              47        Lot-et-Garonne
8              64  Pyrénées-Atlantiques
9              79           Deux-Sèvres
10             86                Vienne
11             87          Haute-Vienne


### Liste des communes de plus de X habitants d'un département donné

In [41]:
# Code du département et nombre minimum d'habitants
id_departement = "24"
min_population = 10_000

request = f"""
SELECT c.id_commune, c.nom_commune, sp.valeur AS population
FROM commune c
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE c.id_departement = '{id_departement}'
  AND sp.annee = 2020
  AND sp.type_statistique = 'Population'
  AND sp.valeur > {min_population};
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_commune', 'nom_commune', 'population'])
print(df)


  id_commune            nom_commune  population
0      24037               Bergerac     26360.0
1      24053  Boulazac Isle Manoire     10738.0
2      24322              Périgueux     29255.0


### La région la plus peuplé

In [42]:
request = """
SELECT r.id_region, r.nom_region, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'population_totale'])
print(df)


   id_region     nom_region  population_totale
0         11  Île-de-France         14417700.0


### Le département le plus peuplé d'une région donnée

In [43]:
nom_region = 'Occitanie'

request = f"""SELECT d.id_departement, d.nom_departement, SUM(sp.valeur) AS population_totale
FROM departement d
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE d.id_region = (SELECT id_region FROM region WHERE nom_region = '{nom_region}')
  AND sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY d.id_departement, d.nom_departement
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement', 'population_totale'])
print(df)

  id_departement nom_departement  population_totale
0             31   Haute-Garonne          1415757.0


### Le nombre de mariages par région 

In [44]:
request = f"""SELECT r.id_region, r.nom_region, SUM(sm.nb_mariages) AS total_mariages
FROM region r
JOIN statistiques_mariages_age sm ON r.id_region = sm.id_region
WHERE sm.annee = 2021
GROUP BY r.id_region, r.nom_region
ORDER BY total_mariages DESC;"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'total_mariages'])
print(df)

    id_region                  nom_region  total_mariages
0          11               Île-de-France           84316
1          84        Auvergne-Rhône-Alpes           53594
2          76                   Occitanie           38690
3          93  Provence-Alpes-Côte d'Azur           37734
4          32             Hauts-de-France           37596
5          75          Nouvelle-Aquitaine           37266
6          44                   Grand Est           35724
7          52            Pays de la Loire           24304
8          28                   Normandie           21306
9          53                    Bretagne           19930
10         27     Bourgogne-Franche-Comté           17230
11         24         Centre-Val de Loire           15838
12         94                       Corse            2236


### Le nombre de mariages par tranche d'âge dans une région donnée

In [45]:
nom_region = 'Nouvelle-Aquitaine'
request = f"""SELECT sma.grage AS tranche_age, SUM(sma.nb_mariages) AS total_mariages
FROM statistiques_mariages_age sma
JOIN region r ON sma.id_region = r.id_region
WHERE sma.annee = 2021 AND r.nom_region = '{nom_region}'
GROUP BY sma.grage
ORDER BY sma.grage;
"""

cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['tranche_age', 'total_mariages'])
print(df)

  tranche_age  total_mariages
0       14_19              48
1       20_24            1410
2       25_29            5593
3       30_34            8148
4       35_39            6024
5       40_49            7401
6       50_59            5135
7       60_PL            3507


### Le nombre de mariages où au moins un des conjoints est un étranger, par région en une année donnée

In [46]:
request = """SELECT r.id_region, r.nom_region, SUM(sme.nb_mariages_nationalite) AS total_mariages
FROM region r
JOIN statistiques_mariages_origine sme ON r.id_region = sme.id_region
WHERE sme.annee = 2020 AND sme.code IN ('FR_ETR', 'ETR_ETR')
GROUP BY r.id_region, r.nom_region
ORDER BY total_mariages DESC;"""
cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'total_mariages'])
print(df)

Empty DataFrame
Columns: [id_region, nom_region, total_mariages]
Index: []


## Views

Créer deux vues (cf commande CREATE OR REPLACE VIEW) qui donnent la population des départements et des régions pour les différentes années ainsi que les indicateurs existants.

### Vue 1 : Population des départements

In [47]:
vue = """
CREATE OR REPLACE VIEW vue_population_departement AS
SELECT d.id_departement, d.nom_departement, sp.annee, sp.type_statistique, SUM(sp.valeur) AS population_totale
FROM departement d
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.type_statistique = 'Population'
GROUP BY d.id_departement, d.nom_departement, sp.annee, sp.type_statistique;
"""

cur.execute(vue)

request = """
SELECT *
FROM vue_population_departement
WHERE annee = 2020
ORDER BY population_totale DESC;
"""
cur.execute(request)
conn.commit()
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement', 'annee', 'type_statistique', 'population_totale'])
df


,id_departement,nom_departement,annee,type_statistique,population_totale
0,75,Paris,2020,Population,4291812.0
1,13,Bouches-du-Rhône,2020,Population,2918391.0
2,59,Nord,2020,Population,2607746.0
3,69,Rhône,2020,Population,2405665.0
4,93,Seine-Saint-Denis,2020,Population,1655422.0
...,...,...,...,...,...
91,15,Cantal,2020,Population,144379.0
92,05,Hautes-Alpes,2020,Population,140605.0
93,90,Territoire de Belfort,2020,Population,140120.0
94,23,Creuse,2020,Population,115995.0


### Vue 2 : Population des régions

In [48]:
vue_region = """
CREATE OR REPLACE VIEW vue_population_region AS
SELECT r.id_region, r.nom_region, sp.annee, sp.type_statistique, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region, sp.annee, sp.type_statistique;
"""

try:
    cur.execute(vue_region)
    conn.commit()  
    
    request = """
    SELECT *
    FROM vue_population_region
    WHERE annee = 2020
    ORDER BY population_totale DESC;
    """
    
    cur.execute(request)
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns=['id_region', 'nom_region', 'annee', 'type_statistique', 'population_totale'])
    print(df)
    
except Exception as e:
    conn.rollback() 
    print(f"Error: {e}")


    id_region                  nom_region  annee type_statistique  \
0          11               Île-de-France   2020       Population   
1          84        Auvergne-Rhône-Alpes   2020       Population   
2          75          Nouvelle-Aquitaine   2020       Population   
3          32             Hauts-de-France   2020       Population   
4          76                   Occitanie   2020       Population   
5          93  Provence-Alpes-Côte d'Azur   2020       Population   
6          44                   Grand Est   2020       Population   
7          52            Pays de la Loire   2020       Population   
8          53                    Bretagne   2020       Population   
9          28                   Normandie   2020       Population   
10         27     Bourgogne-Franche-Comté   2020       Population   
11         24         Centre-Val de Loire   2020       Population   
12         94                       Corse   2020       Population   

    population_totale  
0        

## Procédure stockée

In [49]:
alter_departements = "ALTER TABLE departement ADD COLUMN population_totale INT;"
alter_regions = "ALTER TABLE region ADD COLUMN population_totale INT;"

cur.execute(alter_departements)
cur.execute(alter_regions)
conn.commit()

# Créer la procédure stockée
procedure_calcul = """
CREATE OR REPLACE PROCEDURE calculer_population()
LANGUAGE plpgsql
AS $$
BEGIN
    -- Calculer la population des départements
    UPDATE departement d
    SET population_totale = sub.population
    FROM (
        SELECT c.id_departement, SUM(sp.valeur) AS population
        FROM commune c
        JOIN statistiques_population sp ON c.id_commune = sp.codgeo
        WHERE sp.type_statistique = 'Population' AND sp.annee = 2020
        GROUP BY c.id_departement
    ) AS sub
    WHERE d.id_departement = sub.id_departement;

    -- Calculer la population des régions
    UPDATE region r
    SET population_totale = sub.population
    FROM (
        SELECT d.id_region, SUM(d.population_totale) AS population
        FROM departement d
        GROUP BY d.id_region
    ) AS sub
    WHERE r.id_region = sub.id_region;
END;
$$;
"""

cur.execute(procedure_calcul)
conn.commit()

# Exécuter la procédure stockée
cur.execute("CALL calculer_population();")
conn.commit()

# Fermeture de la connexion
cur.close()
conn.close()

In [50]:
# On verifie que les colonnes ont bien été ajoutées et que les populations ont bien été calculées
conn = connect()
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

request = """
SELECT * FROM departement;
"""
cur.execute(request)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id_departement', 'nom_departement', 'id_region', 'population_totale'])
print(df)

   id_departement          nom_departement  id_region  population_totale
0              01                      Ain         84             657856
1              02                    Aisne         32             529374
2              03                   Allier         84             335628
3              04  Alpes-de-Haute-Provence         93             165451
4              05             Hautes-Alpes         93             140605
..            ...                      ...        ...                ...
91             91                  Essonne         11            1306118
92             92           Hauts-de-Seine         11            1626213
93             93        Seine-Saint-Denis         11            1655422
94             94             Val-de-Marne         11            1407972
95             95               Val-d'Oise         11            1251804

[96 rows x 4 columns]


## Triggers

In [51]:
try:
    # Création du trigger pour empêcher les modifications dans la table region
    bloque_region = """
    CREATE OR REPLACE FUNCTION bloque_modifications_region()
    RETURNS trigger AS $$
    BEGIN
        RAISE EXCEPTION 'Modification de la table region non autorisée';
        RETURN NULL;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_bloquer_modifications_region
    BEFORE INSERT OR UPDATE OR DELETE ON region
    FOR EACH ROW EXECUTE FUNCTION bloque_modifications_region();
    """
    
    # Création du trigger pour empêcher les modifications dans la table departement
    bloque_departement = """
    CREATE OR REPLACE FUNCTION bloque_modifications_departement()
    RETURNS trigger AS $$
    BEGIN
        RAISE EXCEPTION 'Modification de la table departement non autorisée';
        RETURN NULL;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_bloquer_modifications_departement
    BEFORE INSERT OR UPDATE OR DELETE ON departement
    FOR EACH ROW EXECUTE FUNCTION bloque_modifications_departement();
    """
    
    # Création de la procédure pour mettre à jour les populations
    procedure_calcul = """
    CREATE OR REPLACE PROCEDURE calculer_population()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        -- Calculer la population des départements
        UPDATE departement d
        SET population_totale = sub.population
        FROM (
            SELECT c.id_departement, SUM(sp.valeur) AS population
            FROM commune c
            JOIN statistiques_population sp ON c.id_commune = sp.codgeo
            WHERE sp.type_statistique = 'Population' AND sp.annee = 2020
            GROUP BY c.id_departement
        ) AS sub
        WHERE d.id_departement = sub.id_departement;

        -- Calculer la population des régions
        UPDATE region r
        SET population_totale = sub.population
        FROM (
            SELECT d.id_region, SUM(d.population_totale) AS population
            FROM departement d
            GROUP BY d.id_region
        ) AS sub
        WHERE r.id_region = sub.id_region;
    END;
    $$;
    """
    
    # Création du trigger pour mettre à jour les populations
    maj_population = """
    CREATE OR REPLACE FUNCTION maj_population()
    RETURNS trigger AS $$
    BEGIN
        PERFORM calculer_population();
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_maj_population
    AFTER INSERT OR UPDATE ON statistiques_population
    FOR EACH ROW EXECUTE FUNCTION maj_population();
    """

    # Exécuter les commandes
    cur.execute(bloque_region)
    cur.execute(bloque_departement)
    cur.execute(procedure_calcul)
    cur.execute(maj_population)
    
    # Valider les changements
    conn.commit()

except Exception as e:
    # Annuler la transaction en cas d'erreur
    conn.rollback()
    print(f"Error: {e}")

finally:
    # Fermer la connexion
    cur.close()
    conn.close()


## Triggers suite

In [52]:
conn = connect()
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

try:
    # Création de la procédure pour mettre à jour la population d'un département
    procedure_update_departement = """
    CREATE OR REPLACE PROCEDURE update_population_departement()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        UPDATE departement d
        SET population_totale = sub.population
        FROM (
            SELECT c.id_departement, SUM(sp.valeur) AS population
            FROM commune c
            JOIN statistiques_population sp ON c.id_commune = sp.codgeo
            WHERE sp.type_statistique = 'Population' AND sp.annee IN (2020, 2021, 2022, 2023)
            GROUP BY c.id_departement
            HAVING COUNT(c.id_commune) = (
                SELECT COUNT(*)
                FROM commune
                WHERE id_departement = c.id_departement
            )
        ) AS sub
        WHERE d.id_departement = sub.id_departement;
    END;
    $$;
    """

    # Création de la procédure pour mettre à jour la population d'une région
    procedure_update_region = """
    CREATE OR REPLACE PROCEDURE update_population_region()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        UPDATE region r
        SET population_totale = sub.population
        FROM (
            SELECT d.id_region, SUM(d.population_totale) AS population
            FROM departement d
            GROUP BY d.id_region
            HAVING COUNT(d.id_departement) = (
                SELECT COUNT(*)
                FROM departement
                WHERE id_region = d.id_region
            )
        ) AS sub
        WHERE r.id_region = sub.id_region;
    END;
    $$;
    """

    # Création de la procédure qui met à jour les populations des départements et des régions
    procedure_update_all = """
    CREATE OR REPLACE PROCEDURE update_population_all()
    LANGUAGE plpgsql
    AS $$
    BEGIN
        PERFORM update_population_departement();
        PERFORM update_population_region();
    END;
    $$;
    """

    # Création du trigger pour mettre à jour les populations
    trigger_update_population = """
    CREATE OR REPLACE FUNCTION trigger_population_update()
    RETURNS trigger AS $$
    BEGIN
        PERFORM update_population_all();
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    CREATE TRIGGER tr_update_population
    AFTER INSERT OR UPDATE ON statistiques_population
    FOR EACH ROW EXECUTE FUNCTION trigger_population_update();
    """

    # Exécution des commandes
    cur.execute(procedure_update_departement)
    cur.execute(procedure_update_region)
    cur.execute(procedure_update_all)
    cur.execute(trigger_update_population)

    # Valider les changements
    conn.commit()

except Exception as e:
    # Annuler la transaction en cas d'erreur
    conn.rollback()
    print(f"Error: {e}")

finally:
    cur.close()
    conn.close()


## Plan d'exécution (EXPLAIN)
### La région la plus peuplé

In [53]:
conn = connect()
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

request = """
EXPLAIN ANALYSE SELECT r.id_region, r.nom_region, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
for row in rows:
    print(row[0])

Limit  (cost=15109.01..15109.01 rows=1 width=82) (actual time=278.208..289.672 rows=1 loops=1)
  ->  Sort  (cost=15109.01..15110.94 rows=770 width=82) (actual time=278.207..289.670 rows=1 loops=1)
        Sort Key: (sum(sp.valeur)) DESC
        Sort Method: top-N heapsort  Memory: 25kB
        ->  Finalize GroupAggregate  (cost=14910.08..15105.16 rows=770 width=82) (actual time=278.151..289.643 rows=13 loops=1)
              Group Key: r.id_region
              ->  Gather Merge  (cost=14910.08..15089.76 rows=1540 width=82) (actual time=278.137..289.619 rows=35 loops=1)
                    Workers Planned: 2
                    Workers Launched: 2
                    ->  Sort  (cost=13910.06..13911.98 rows=770 width=82) (actual time=265.169..265.178 rows=12 loops=3)
                          Sort Key: r.id_region
                          Sort Method: quicksort  Memory: 25kB
                          Worker 0:  Sort Method: quicksort  Memory: 25kB
                          Worker 1:  So

Dans l'output ci-dessus, vous pouvez voir que le coût total estimé pour la requête est de 15108.32 unités. Cependant, le coût réel d'exécution peut être différent, car les estimations de coût sont basées sur les statistiques de la base de données, qui peuvent ne pas être parfaitement précises.

Dans notre cas, le coût réel d'exécution de la requête était d'environ 104,881 ms (Execution Time: 104.881 ms). Cela signifie que la requête a pris un peu plus de 100 millisecondes pour s'exécuter.

On peut voir que PostgreSQL a utilisé différents algorithmes de jointure et de tri pour exécuter la requête.

**Algorithmes de jointure**

1. **Hash Join** : Cet algorithme a été utilisé pour joindre les tables `statistiques_population` et `commune` sur la colonne `id_commune`. Il a également été utilisé pour joindre les tables `commune` et `departement` sur la colonne `id_departement`.
2. **Nested Loop Join** : Cet algorithme a été utilisé pour joindre les tables `departement` et `region` sur la colonne `id_region`.

**Stratégies de tri**

1. **Sort Method: quicksort** : Cette stratégie de tri a été utilisée pour trier les résultats intermédiaires de la jointure entre les tables `statistiques_population`, `commune`, `departement` et `region`.
2. **Top-N heapsort** : Cette stratégie de tri a été utilisée pour trier les résultats finaux de la requête, en utilisant l'ordre de tri spécifié dans la clause `ORDER BY` et en limitant le nombre de résultats à 1.

Le choix de l'algorithme de jointure ou de tri dépend de plusieurs facteurs, tels que la taille des tables, la présence ou non d'index sur les colonnes de jointure, et les contraintes de performance spécifiées dans la requête. Dans ce cas, PostgreSQL a choisi les algorithmes et les stratégies de tri les plus efficaces pour exécuter la requête en fonction des données et des contraintes spécifiées. 

Afin de montrer l'intérêt des index, nous allons en implementer quelques-uns avant de comparer leurs performances.

## Plan d'exécution et index

In [54]:
request = """
CREATE INDEX pop_idx ON statistiques_population (valeur) WHERE type_statistique = 'Population';
EXPLAIN ANALYSE SELECT r.id_region, r.nom_region, SUM(sp.valeur) AS population_totale
FROM region r
JOIN departement d ON r.id_region = d.id_region
JOIN commune c ON d.id_departement = c.id_departement
JOIN statistiques_population sp ON c.id_commune = sp.codgeo
WHERE sp.annee = 2020 AND sp.type_statistique = 'Population'
GROUP BY r.id_region, r.nom_region
ORDER BY population_totale DESC
LIMIT 1;
"""

cur.execute(request)
rows = cur.fetchall()
for row in rows:
    print(row[0])

Limit  (cost=12569.27..12569.27 rows=1 width=82) (actual time=108.643..108.653 rows=1 loops=1)
  ->  Sort  (cost=12569.27..12571.20 rows=770 width=82) (actual time=108.642..108.652 rows=1 loops=1)
        Sort Key: (sum(sp.valeur)) DESC
        Sort Method: top-N heapsort  Memory: 25kB
        ->  HashAggregate  (cost=12557.72..12565.42 rows=770 width=82) (actual time=108.626..108.641 rows=13 loops=1)
              Group Key: r.id_region
              Batches: 1  Memory Usage: 49kB
              ->  Hash Join  (cost=2790.40..12413.60 rows=28824 width=82) (actual time=25.776..100.571 rows=34861 loops=1)
                    Hash Cond: (d.id_region = r.id_region)
                    ->  Hash Join  (cost=2763.07..12310.20 rows=28824 width=12) (actual time=25.745..88.399 rows=34861 loops=1)
                          Hash Cond: ((c.id_departement)::text = (d.id_departement)::text)
                          ->  Hash Join  (cost=2758.91..12227.04 rows=28824 width=11) (actual time=25.698..74.91

On peut voir que l'utilisation d'un index a permis d'améliorer les performances de la requête.

Sans l'index, la requête a dû parcourir l'intégralité de la table `statistiques_population` pour trouver les lignes correspondant à la condition `type_statistique = 'Population'`. Cela aurait nécessité un séquentiel scan complet de la table, ce qui peut être coûteux en termes de temps et de ressources, en particulier si la table est volumineuse.

Avec l'index, la requête a pu utiliser un bitmap index scan pour trouver rapidement les lignes correspondant à la condition `type_statistique = 'Population'`. Cela a permis de réduire considérablement le nombre de lignes à examiner, ce qui a accéléré la requête.

On peut voir dans l'output que le bitmap index scan a pris seulement 11,554 ms pour renvoyer les 104 583 lignes correspondant à la condition, alors qu'un séquentiel scan complet de la table aurait pris beaucoup plus de temps.

Ensuite, la requête a utilisé un hash join pour joindre les tables `statistiques_population`, `commune`, `departement` et `region`, en utilisant les clés primaires et étrangères pour relier les lignes correspondantes. L'utilisation d'un index sur la colonne `id_commune` de la table `statistiques_population` a permis d'accélérer la jointure en utilisant un hash join plus efficace.

Enfin, la requête a utilisé un tri top-N heapsort pour trier les résultats finaux en fonction de la population totale, et a renvoyé le premier résultat en utilisant une limite de 1.

En comparant cet output à celui sans index, on peut voir que l'utilisation d'index a permis d'améliorer considérablement les performances de la requête, en réduisant le temps d'exécution de plus de la moitié (de 290.121 ms à 109.404 ms). Cela démontre l'importance des index dans l'optimisation des requêtes SQL.

## Pour aller plus loin : transactions